In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
from scipy.sparse import csr_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingRegressor
from multiprocessing import Pool
from sklearn.metrics import mean_squared_error

In [2]:
df_train = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_train['date'] = pd.to_datetime(df_train['date'], format='%d.%m.%Y')

### Let's submit prev month values
score: 1.16778 
let it be baseline

In [28]:
df_last_month_groupped = df_train[df_train['date_block_num'] == 33].groupby(['shop_id', 'item_id'])['item_cnt_day'].sum()
df_test_submission = df_test.set_index(['shop_id', 'item_id'])
df_test_submission = df_test_submission.join(df_last_month_groupped)
df_test_submission = df_test_submission.fillna(0)
df_test_submission['item_cnt_day'] = df_test_submission['item_cnt_day'].clip(upper=20)
df_test_submission = df_test_submission.reset_index()
df_test_submission = df_test_submission[['ID', 'item_cnt_day']]

df_test_submission.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
df_test_submission.to_csv('advice2.csv', index=False)

In [3]:

start = time.time()
df_iter = df_train.copy()

df_out = df_iter.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().to_frame()
base = df_out.reset_index()

for lag in range(1, 6):
    base_group_shift = base.groupby(['shop_id', 'item_id'])\
                       .apply(lambda x: x.sort_values(by='date_block_num')['item_cnt_day'].shift(lag)).reset_index(drop=True)
    base_group = base.copy()
    base_group['item_cnt_day'] = base_group_shift
    df_out = pd.merge(df_out, 
                  base_group, 
                  left_index=True, right_on=['shop_id', 'item_id', 'date_block_num'], suffixes=('',f'_{lag}'))
    df_out.set_index(['shop_id', 'item_id', 'date_block_num'], inplace=True)
print('elapsed {}'.format(time.time()-start))
df_out.to_csv('train_5_lags.csv')


# !Note theris no previuos date_block_num 0 because it is missing

elapsed 1256.2061812877655


In [164]:
print(pd.util.hash_pandas_object(df_out).sum())

-8855404477981348569


In [ ]:
start = time.time()
df_pooled = df_train.copy()
pooled_base_group = df_pooled.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().to_frame().reset_index()
n_cores = 4
split_index = np.array_split(pooled_base_group[['shop_id', 'item_id']].drop_duplicates(), n_cores)
split_df = [pd.merge(index, pooled_base_group, left_on=['shop_id', 'item_id'], right_on=['shop_id', 'item_id'], how='inner')\
            .set_index(['shop_id', 'item_id', 'date_block_num'])\
            for index in split_index]
print(split_df[0])
def create_lags(df):
    base = df.copy()

    for lag in range(1, 6):
        df[f'item_cnt_day_{lag}'] = base.groupby(['shop_id', 'item_id'])\
                           .apply(lambda x: x.sort_values(by='date_block_num')['item_cnt_day'].shift(lag))\
            .reset_index(drop=True).values
#         base_group = base.copy()
#         df[f'item_cnt_day_{lag}'] = base_group_shift
#         df = pd.merge(df, 
#                       base_group, 
#                       left_index=True, right_index=True, suffixes=('',f'_{lag}'))
#         df.set_index(['shop_id', 'item_id', 'date_block_num'], inplace=True)
    return df

pool = Pool(n_cores)
df = pd.concat(pool.map(create_lags, split_df))
pool.close()
pool.join()


print('elapsed multithereaded {}'.format(time.time()-start))


                                item_cnt_day
shop_id item_id date_block_num              
0       30      1                       31.0
        31      1                       11.0
        32      0                        6.0
                1                       10.0
        33      0                        3.0
...                                      ...
18      7534    30                       1.0
                33                       1.0
        7536    0                        1.0
                1                        3.0
                3                        1.0

[335572 rows x 1 columns]


In [16]:
def create_XY_matrix(df_5_lags, shop_id_to_feature_id, item_id_to_feature_id, other_feature_to_feature_id):
    df_5_lags.reset_index(inplace=True)
    df_5_lags.fillna(0, inplace=True)
    cols = []
    rows = []
    
    shop_feature_count = len(shop_id_to_feature_id)
    print(f'shop_feature_count {shop_feature_count}')
    item_feature_count = len(item_id_to_feature_id)
    print(f'item_feature_count {item_feature_count}')    
    shop_and_item_feature_count = shop_feature_count + item_feature_count
    
    for i, row in df_5_lags.iterrows():
        rows.append(i)
        cols.append(shop_id_to_feature_id[row['shop_id']])
        rows.append(i)
        cols.append(item_id_to_feature_id[row['item_id']])
    
    data = [1] * len(cols)    
    
    print(other_feature_to_feature_id)
    
    for i, row in df_5_lags.iterrows():
        for other_column in rest_columns:
            rows.append(i)
            cols.append(other_feature_to_feature_id[other_column])
            data.append(row[other_column])
    rows_total = df_5_lags.shape[0]
    colls_total = shop_and_item_feature_count + len(other_feature_to_feature_id)
    print(f'rows_total {rows_total}, colls_total {colls_total}')
    print('rows max {}'.format(max(rows)))
    return csr_matrix((data, (rows, cols)), shape=(rows_total, colls_total)),\
           df_5_lags['item_cnt_day'].values

def split_train_test(df, shop_id_to_feature_id, item_id_to_feature_id, other_feature_to_feature_id):
    max_data_block = df['date_block_num'].max()    
    trainX, trainY = create_XY_matrix(df[df['date_block_num'] != max_data_block],
                                     shop_id_to_feature_id, item_id_to_feature_id, other_feature_to_feature_id)
    testX, testY = create_XY_matrix(df[df['date_block_num'] == max_data_block],
                                    shop_id_to_feature_id, item_id_to_feature_id, other_feature_to_feature_id)
    return trainX, trainY, testX, testY

df_lagged = pd.read_csv('train_5_lags.csv')

shop_id_to_feature_id = {v:i for i, v in enumerate(df_lagged['shop_id'].unique())}
item_id_to_feature_id = {v: i + len(shop_id_to_feature_id) for i, v in enumerate(df_lagged['item_id'].unique())}
rest_columns = np.array(df_lagged.columns)
rest_columns = np.delete(rest_columns, 
                        np.argwhere((rest_columns=='shop_id') | (rest_columns=='item_id') | (rest_columns=='item_cnt_day')))
other_feature_to_feature_id = {v: i + len(shop_id_to_feature_id) + len(item_id_to_feature_id) 
                               for i, v in enumerate(rest_columns)}

trainX, trainY, testX, testY = split_train_test(df_lagged, 
                                                shop_id_to_feature_id, 
                                                item_id_to_feature_id, 
                                                other_feature_to_feature_id)


C:\Users\aimar\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


shop_feature_count 60
item_feature_count 21807
{'date_block_num': 21867, 'item_cnt_day_1': 21868, 'item_cnt_day_2': 21869, 'item_cnt_day_3': 21870, 'item_cnt_day_4': 21871, 'item_cnt_day_5': 21872}
rows_total 1577593, colls_total 21873
rows max 1577592
shop_feature_count 60
item_feature_count 21807
{'date_block_num': 21867, 'item_cnt_day_1': 21868, 'item_cnt_day_2': 21869, 'item_cnt_day_3': 21870, 'item_cnt_day_4': 21871, 'item_cnt_day_5': 21872}
rows_total 31531, colls_total 21873
rows max 31530


In [26]:
est = GradientBoostingRegressor(n_estimators=10, learning_rate=0.01,
    max_depth=10, random_state=0, loss='ls').fit(trainX, trainY)

predicted = est.predict(testX)
print('RMSE: {}'.format(mean_squared_error(testY, predicted, squared=False)))


RMSE: 14.110279933439532


### plan: use advice #2 and #3

Create lags and mean encoding

In [31]:
# 1. uber aggregation group

# prepare date
df_train['date'] = pd.to_datetime(df_train['date'], format='%d.%m.%Y')

# group by item, shop and sort by date 
# we should not getting window out of this groupping!

df_iter = df_train.copy()
df_iter.set_index(['shop_id', 'item_id'], inplace=True)
df_output = pd.DataFrame()
for index in df_iter.index.values:
    shop_id = index[0]
    item_id = index[1]
    group = df_iter.loc[(shop_id, item_id)]
    group.sort_values('date', inplace=True)
    rolling = group['item_cnt_day'].rolling(6, 1).mean()
    for lag in range(1,6):
        group[f'item_cnt_day_lag{lag}'] = rolling.shift(lag)
    df_output = pd.concat([df_output, group])
# df_output
    
#     size = df.shape[0]
#     print(f' {i}: {shop_id} {item_id}, size={size}')
#     i = i + 1


# .apply(lambda x: x.sort_values('date'))

# rolling within 6 periods
# item_cnt_day_moving_avg = df_item_grouping['item_cnt_day'].transform(lambda x: x.rolling(6, 1).mean())

# df_train_lagged = df_item_grouping['item_cnt_day'].sum().reset_index()

# df_train_lagged

C:\Users\aimar\anaconda3\lib\site-packages\ipykernel_launcher.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app
C:\Users\aimar\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\aimar\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 

In [27]:
%%time
pd.read_csv('sales_train.csv')

Wall time: 1.07 s


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [15]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

df = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4], 'col3':[5,6]})
df['col1'] = df['col1'].astype('category')
df['col2'] = df['col2'].astype('category')
df.info()
# enc = OneHotEncoder(handle_unknown='ignore')
# enc.fit([[1,2], [3,4]])
# print(enc.categories_)
# m=enc.transform(df)
# print(m.toarray())

type(pd.get_dummies(df, sparse=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   col1    2 non-null      category
 1   col2    2 non-null      category
 2   col3    2 non-null      int64   
dtypes: category(2), int64(1)
memory usage: 340.0 bytes


pandas.core.frame.DataFrame

In [4]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
np.concatenate((a, b), axis=1)

array([[1, 2, 5, 6],
       [3, 4, 7, 8]])